# Checking your fitting results

## Checking your optimisation set-up with synthetic data studies

## Checking your real-data results with repeated fits

sing the boundaries-with-sampling object defined above, we can run fitting experiments repeatedly, from different starting points:

1. Sample $x_0[i]$ from within the boundaries.
2. Run the i-th optimisation, starting from $x_0[i]$.
3. Store the resulting lowest error $f_\text{opt}[i]$ and best position $x_\text{opt}[i]$.

When working with real data, where the "true" solution is unknown, repeated fits are crucial to test the reliability of the obtained results.

We recommend the following strategy:

1. Run at least 50 repeats, and plot the results from each in a boundary plot as shown above.
2. Identify the repeat $i_\text{best}$ with the lowest error $f_\text{opt}[i_\text{best}]$
3. Define some distance measure, e.g. euclidean distance in a space with log-transformed alpha parameters, and calculate the distance $d(x_\text{opt}[i], x_\text{opt}[i_\text{best}])$ for each of the 50 repeats

For a succesful optimisation, this should result in a distribution where a large proportion of the distances (e.g. 80\%) are close to 0.
A very low number of distances near zero (e.g. less than 5) can indicate that the "best" result was very difficult to find, but could also show that the best result found was not a global optimum.
If this occurs, either the optimisation needs to be improved somehow, or the number of repeats should be increased.

## Testing the usefulness of your model: training and validation data

## Summary

In this notebook, we

In the next notebook